Nosso problema é um problema de classificação de multi-label em que pretendemos prever as tags relevantes para uma determinada pergunta com base em seu título e corpo. O objetivo é atribuir uma ou mais tags a cada pergunta com precisão. Essa é uma tarefa essencial para plataformas on-line de resposta a perguntas, como o Stack Overflow, em que milhões de perguntas são publicadas diariamente, e torna-se cada vez mais difícil para os usuários encontrar perguntas relevantes para responder ou ler.

O problema é desafiador porque uma única pergunta pode ter várias tags relevantes, o que a torna um problema de classificação de multi-label. Além disso, as perguntas no Stack Overflow podem ser altamente técnicas, e a mesma palavra pode ter significados diferentes dependendo do contexto, o que torna difícil prever com precisão as tags relevantes. Portanto, precisamos pré-processar os dados, identificar os recursos importantes e usar técnicas adequadas de aprendizado de máquina para resolver esse problema com precisão.

Os insights obtidos com esse problema podem ajudar a melhorar a experiência do usuário em plataformas on-line de perguntas e respostas, sugerindo tags relevantes para a pergunta, o que facilita para os usuários encontrarem as respostas de que precisam. Além disso, ele pode ajudar as comunidades on-line a organizar e categorizar melhor suas perguntas, facilitando a navegação por elas.

In [ ]:
# O problema que estamos tentando resolver é o de predição de tags para perguntas do StackOverflow. 

In [55]:
#%pip install contractions

In [56]:
import pandas as pd 

In [57]:
# carregando os dados
questions_df = pd.read_csv('data/Questions.csv', encoding="ISO-8859-1", usecols =['Id','Score', 'Title', 'Body'])
questions_df = questions_df.sample(n=20000)

In [58]:
# Cleaning the data
import re
import string
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

def normalize_text(s):
    s = s.lower()
    return s

def remove_html_tags(text):
    text = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
    text = re.sub('<code>.*?</code>', '', text, flags=re.DOTALL)
    text = re.sub('<[^>]+>', '', text, flags=re.DOTALL)
    return text.replace("\n", "")

def remove_punctuation(text):
    res = text.split()
    for word in res:
        if word in string.punctuation:
            res.remove(word)
    return ' '.join(res) 

def remove_contractions(text):
    return contractions.fix(text)

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text)])

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([w for w in word_tokenize(text) if not w in stop_words])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danilo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
questions_df['Body'] = questions_df['Body'].apply(remove_html_tags)
questions_df['Body'] = questions_df['Body'].apply(remove_contractions)
questions_df['Body'] = questions_df['Body'].apply(normalize_text)
questions_df['Body'] = questions_df['Body'].apply(remove_stopwords)
questions_df['Body'] = questions_df['Body'].apply(remove_punctuation)
questions_df['Body'] = questions_df['Body'].apply(lemmatize)
questions_df['Title'] = questions_df['Title'].apply(remove_html_tags)
questions_df['Title'] = questions_df['Title'].apply(remove_contractions)
questions_df['Title'] = questions_df['Title'].apply(normalize_text)
questions_df['Title'] = questions_df['Title'].apply(remove_stopwords)
questions_df['Title'] = questions_df['Title'].apply(remove_punctuation)
questions_df['Title'] = questions_df['Title'].apply(lemmatize)

In [60]:
# Print the first rows of `questions_df`
print(questions_df.head())

              Id  Score                                              Title   
724361  24625620      3  http server respond head request chunked encoding  \
735150  24964840      0                  io uitextview wrapping every line   
7232      490040      0                 nhibernate evicting object session   
469140  16411980      1  calling angular controller function external j...   
882884  29560870      3                             azure busy worker role   

                                                     Body  
724361  question http server response look like head s...  
735150  pretty sure data source funky \p element seein...  
7232    mapping productcategory tree using fluent nhib...  
469140  user click word displaypopup ) called create a...  
882884  microsoft azure project project deploy 3 cloud...  


In [61]:

def join_strings(x):
    return " ".join(x)


# Load the tags dataset
tags_df = pd.read_csv("data/Tags.csv", encoding="ISO-8859-1", dtype={'Tag': str})
tags_df['Tag'] = tags_df['Tag'].astype(str)

tags_group = tags_df.groupby("Id")['Tag'].apply(join_strings)
tags = pd.DataFrame({'Id':tags_group.index, 'Tags':tags_group.values})


In [62]:
tags

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction
...,...,...
1264211,40143210,php .htaccess
1264212,40143300,google-bigquery
1264213,40143340,android android-studio
1264214,40143360,javascript vue.js


In [63]:
merged_df = pd.merge(questions_df, tags, on='Id')

In [64]:
# For y we need the order of tags to not affect the result, so we use an encoder to transform the tags into numbers
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(merged_df['Title'] + ' ' + merged_df['Body'])

mlb = MultiLabelBinarizer(sparse_output=True)

y = mlb.fit_transform(merged_df['Tags'].str.split(" "))

In [65]:
# Splitting the data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Training the model for a multilabel classification, using a OneVsRestClassifier with a LogisticRegression and knowing that the data is sparse
# We need the 4 most probable tags for each question, so we use predict_proba instead of predict

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(lr)

clf.fit(X_train, y_train)

C:\Users\Danilo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(
C:\Users\Danilo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 10 is present in all training examples.
  warnings.warn(
C:\Users\Danilo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
C:\Users\Danilo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\multiclass.py:77: UserWarning: Label not 13 is present in all training examples.
  warnings.warn(
C:\Users\Danilo\A

OneVsRestClassifier(estimator=LogisticRegression())

In [66]:
y_pred = clf.predict_proba(X_test)

In [105]:
# Display the 4 most probable tags for each question, store the predicted tags and compare them with the real tags
import numpy as np

# We need to sort the tags by probability in descending order
real_tags = mlb.inverse_transform(y_test)
tags_index_probability = np.argsort(y_pred, axis=1)[:, -4:]
predicted_tags = mlb.classes_[tags_index_probability]

for i in range(10):
    print("Title:\n", merged_df['Title'].iloc[i])
    print("Body:\n", merged_df['Body'].iloc[i])
    print("Predicted tags:\n", predicted_tags[i])
    print("Real tags:\n", real_tags[i])
    print("\n")

Title:
 http server respond head request chunked encoding
Body:
 question http server response look like head sent resource server decided perform chunked encoding server always wish perform chunked encoding get specific resource know exact content-length generating response server behave head sent resource
Predicted tags:
 ['java' 'c++' 'c#' 'android']
Real tags:
 ('xamarin',)


Title:
 io uitextview wrapping every line
Body:
 pretty sure data source funky \p element seeing lot wrapping text uitextview apparent reason text wrap console missing something uitextview property
Predicted tags:
 ['python' 'mysql' 'sql-server' 'sql']
Real tags:
 ('sqlalchemy', 'sql', 'database')


Title:
 nhibernate evicting object session
Body:
 mapping productcategory tree using fluent nhibernate everything going fine tried walk tree returned database ensure saving retreiving appropriately.here testing instantiate 4 category beverage beer light beer dark beeradd beer beverage light beer dark beer beer.save

In [115]:
# Find the index of each tag in real tags in the predicted tag, taking into account that the tags are sorted by probability
def find_index(real_tags, predicted_tags):
    indexes = []
    for i in range(len(real_tags)):
        indexes.append([])
        for tag in real_tags[i]:
            res = np.where(predicted_tags[i] == tag)
            if res[0].size > 0:
                indexes[i].append(res[0][0])
    return indexes

# Calculate the accuracy of the model
def accuracy(real_tags, predicted_tags):
    indexes = find_index(real_tags, predicted_tags)
    #calculate the accuracy by how many tags were predicted correctly
    return sum([1 for i in range(len(indexes)) if len(indexes[i]) > 0]) / len(indexes)


In [119]:
accuracy(real_tags, predicted_tags)

0.7296

In [121]:
# Frequência das tags reais
from collections import Counter

real_tags_list = [tag for tags in real_tags for tag in tags]
Counter(real_tags_list).most_common(10)

[('javascript', 480),
 ('java', 420),
 ('php', 393),
 ('c#', 384),
 ('android', 365),
 ('jquery', 318),
 ('python', 271),
 ('html', 243),
 ('c++', 190),
 ('mysql', 186)]

In [120]:
# Frequência das tags em predicted_tags
from collections import Counter

predicted_tags_list = [tag for tags in predicted_tags for tag in tags]
Counter(predicted_tags_list).most_common(10)

[('c#', 2869),
 ('java', 2791),
 ('javascript', 2558),
 ('php', 1493),
 ('android', 1128),
 ('c++', 1036),
 ('jquery', 854),
 ('html', 825),
 ('python', 800),
 ('ios', 621)]

A acurácia é testada com as 4 tags mais prováveis, mas podemos testar com as 3, 2 ou 1 mais provável. Esse método de acurácia de apliquei se remete a um tipo de acurácia que é usada em problemas de classificação multiclasse, onde se considera que o modelo acertou se ele acertou pelo menos uma das classes
Esse tipo de teste é mais adequado para o nosso problema, pois se o modelo acertar pelo menos uma das tags, já é um bom resultado, o usuário pode ter por exemplo sugestões de tags para completar a pergunta no stack oveflow, e não necessariamente todas as tags que ele queria.

Mesmo assim, seria melhor a acurácia ser mais próximo do 100%, o fato de que o calculo da acurácia ser calculado pela presença de pelo menos uma tag correta já infla o numero de acerto, e para esse tipo era esperado algo perto da perfeição. Por isso há algumas suspeitas sobre o que pode ter causado baixa precisão, por exemplo dados desbalanceados, se o número de exemplos para cada tag for desbalanceado, o modelo poderá não aprender a prever com precisão as tags menos frequentes. Isso pode levar a uma baixa precisão para essas tags. E no nosso caso, pegando apenas uma porcentagem pequena (por motivos de performance) acarretou a varias tags não serem inclusas no treino ou nos testes. Ambiguidade nos dados, pode ser difícil fazer tags com precisão em algumas perguntas, pois elas estão relacionadas a vários tópicos. Nesses casos, o modelo pode ter dificuldade para escolher a tag correta.

Com base na análise da frequência das tags, podemos ver que as tags mais comuns nos dados reais são javascript, java, php, c# e android. Por outro lado, nas tags preditas, vemos que as tags mais comuns são c#, java, javascript, php e android. Isso sugere que o modelo é capaz de capturar as tags mais comuns nos dados reais até certo ponto, mas não é capaz de capturar as tags menos frequentes também.
Isso não é surpreendente, pois o modelo foi treinado em um número limitado de tags e é difícil generalizar para tags novas e menos frequentes. Também é possível que o modelo não tenha conseguido captar as nuances das tags menos frequentes devido ao número limitado de exemplos para cada tag nos dados de treinamento.
Entretanto, vale a pena observar que o modelo conseguiu identificar algumas tags menos frequentes, como "nhibernate" e "sqlalchemy", que não estavam entre as tags mais frequentes nos dados reais. Isso sugere que o modelo tem alguma capacidade de generalizar para tags menos frequentes, mas certamente há espaço para melhorias nessa área.
